<a href="https://colab.research.google.com/github/unclepete-20/lab7-k-means/blob/main/K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio #7 (K-Means)

## Integrantes:


*   Pedro Pablo Arriola Jimenez (20188)
*   Oscar Fernando Lopez Barrios (20679)
*   Santiago Taracena Puga (20017)
*   YongBum Park (20117)








# Introducción a K-Means Clustering para Análisis de Transacciones Bancarias 💳💰

El análisis de transacciones bancarias es una tarea importante para detectar fraudes, comportamientos inusuales y patrones de gasto de los clientes. Una forma de analizar estos datos es mediante el uso de técnicas de agrupamiento, como el algoritmo de K-Means.

El algoritmo de K-Means es una técnica de aprendizaje no supervisado que permite agrupar datos en clusters o grupos, basado en su similitud. En el caso del análisis de transacciones bancarias, se pueden agrupar los datos según el comportamiento de los clientes, como sus patrones de gasto, lugares frecuentes de uso de tarjeta, entre otros.

La implementación de K-Means en Python es relativamente sencilla gracias a la disponibilidad de diversas librerías como scikit-learn y pandas. A través del uso de esta técnica, se puede obtener una mejor comprensión de los patrones y comportamientos de los clientes, lo que puede ser valioso para la toma de decisiones en el ámbito bancario.

En esta investigación, se explorará el uso de K-Means Clustering para el análisis de transacciones bancarias, incluyendo su implementación en Python y la interpretación de los resultados obtenidos.


## Task 1 - Limpieza de datos y Análisis Exploratorio

In [1]:
# Librerías necesarias para la limpieza de datos y análisis exploratorio.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Se carga el dataset para comenzar a realizar limpieza y exploración.
data = pd.read_csv("./data/bank_transactions.csv")
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


La tabla muestra información sobre transacciones financieras. A continuación se describe el significado de cada columna:

- TransactionID: un identificador único para cada transacción
- CustomerID: un identificador único para cada cliente
- CustomerDOB: la fecha de nacimiento del cliente
- CustGender: el género del cliente
- CustLocation: la ubicación geográfica del cliente
- CustAccountBalance: el saldo de la cuenta del cliente
- TransactionDate: la fecha de la transacción
- TransactionTime: la hora de la transacción
- TransactionAmount (INR): la cantidad de la transacción en rupias indias (INR).

Con esta información, se procederá con la limpieza y codificación de los datos.

In [53]:
# Se eliminar estas variables categóricas que no sirven ningún propósito.
data = data.drop(["TransactionID", "CustomerID", "CustomerDOB", "TransactionDate"], axis=1)
data.head()

,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,F,JAMSHEDPUR,17819.05,143207,25.0
1,M,JHAJJAR,2270.69,141858,27999.0
2,F,MUMBAI,17874.44,142712,459.0
3,F,MUMBAI,866503.21,142714,2060.0
4,F,NAVI MUMBAI,6714.43,181156,1762.5
...,...,...,...,...,...
1048562,M,NEW DELHI,7635.19,184824,799.0
1048563,M,NASHIK,27311.42,183734,460.0
1048564,M,HYDERABAD,221757.06,183313,770.0
1048565,M,VISAKHAPATNAM,10117.87,184706,1000.0


In [54]:
# Se hace un mapeo para codificar el genero del cliente

gender_map = {"M": 1, "F": 0}  # Mapeo de género a valores numéricos
data["CustGender"] = data["CustGender"].replace(gender_map)
data

,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,0,JAMSHEDPUR,17819.05,143207,25.0
1,1,JHAJJAR,2270.69,141858,27999.0
2,0,MUMBAI,17874.44,142712,459.0
3,0,MUMBAI,866503.21,142714,2060.0
4,0,NAVI MUMBAI,6714.43,181156,1762.5
...,...,...,...,...,...
1048562,1,NEW DELHI,7635.19,184824,799.0
1048563,1,NASHIK,27311.42,183734,460.0
1048564,1,HYDERABAD,221757.06,183313,770.0
1048565,1,VISAKHAPATNAM,10117.87,184706,1000.0


In [55]:
# Se eliminan datos faltantes o nulos
data = data.dropna()
data

,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,0,JAMSHEDPUR,17819.05,143207,25.0
1,1,JHAJJAR,2270.69,141858,27999.0
2,0,MUMBAI,17874.44,142712,459.0
3,0,MUMBAI,866503.21,142714,2060.0
4,0,NAVI MUMBAI,6714.43,181156,1762.5
...,...,...,...,...,...
1048562,1,NEW DELHI,7635.19,184824,799.0
1048563,1,NASHIK,27311.42,183734,460.0
1048564,1,HYDERABAD,221757.06,183313,770.0
1048565,1,VISAKHAPATNAM,10117.87,184706,1000.0


In [56]:
# Escalamiento de la data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[["CustAccountBalance", "TransactionAmount (INR)"]] = scaler.fit_transform(data[["CustAccountBalance", "TransactionAmount (INR)"]])
data

<ipython-input-56-0ed1aeac0f62>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['CustAccountBalance', 'TransactionAmount (INR)']] = scaler.fit_transform(data[['CustAccountBalance', 'TransactionAmount (INR)']])


,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,0,JAMSHEDPUR,-0.115327,143207,-0.235560
1,1,JHAJJAR,-0.133684,141858,4.022061
2,0,MUMBAI,-0.115261,142712,-0.169506
3,0,MUMBAI,0.886689,142714,0.074165
4,0,NAVI MUMBAI,-0.128438,181156,0.028886
...,...,...,...,...,...
1048562,1,NEW DELHI,-0.127351,184824,-0.117758
1048563,1,NASHIK,-0.104119,183734,-0.169354
1048564,1,HYDERABAD,0.125457,183313,-0.122172
1048565,1,VISAKHAPATNAM,-0.124419,184706,-0.087166


In [57]:
median_balance = data["CustAccountBalance"].median()
data.loc[:, "CustAccountBalance"] = data["CustAccountBalance"].fillna(median_balance)
data

<ipython-input-57-2d815548a156>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'CustAccountBalance'] = data['CustAccountBalance'].fillna(median_balance)


,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,0,JAMSHEDPUR,-0.115327,143207,-0.235560
1,1,JHAJJAR,-0.133684,141858,4.022061
2,0,MUMBAI,-0.115261,142712,-0.169506
3,0,MUMBAI,0.886689,142714,0.074165
4,0,NAVI MUMBAI,-0.128438,181156,0.028886
...,...,...,...,...,...
1048562,1,NEW DELHI,-0.127351,184824,-0.117758
1048563,1,NASHIK,-0.104119,183734,-0.169354
1048564,1,HYDERABAD,0.125457,183313,-0.122172
1048565,1,VISAKHAPATNAM,-0.124419,184706,-0.087166


In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data.loc[:, "CustLocation"] = le.fit_transform(data["CustLocation"])
data


<ipython-input-58-cb6380ca4cc4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'CustLocation'] = le.fit_transform(data['CustLocation'])


,CustGender,CustLocation,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,0,3567,-0.115327,143207,-0.235560
1,1,3629,-0.133684,141858,4.022061
2,0,5242,-0.115261,142712,-0.169506
3,0,5242,0.886689,142714,0.074165
4,0,5631,-0.128438,181156,0.028886
...,...,...,...,...,...
1048562,1,5766,-0.127351,184824,-0.117758
1048563,1,5603,-0.104119,183734,-0.169354
1048564,1,3377,0.125457,183313,-0.122172
1048565,1,9108,-0.124419,184706,-0.087166
